In [1]:
import uproot
import numpy as np
import torch

import GraphBuilder

In [2]:
#############################
# Let's read the file
#############################

#fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/files/nue_dune10kt_1x2x6_1124_526_20230828T185125Z_gen_g4_detsim_hitreco_ccnutree'#'/Users/isobel/Desktop/DUNE/2024/Hierarchy/files/ccnutree_higherStats'
#fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/files/ccnutree_higherStats'


fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/files/ccnutree_0_1_2'


inputFileName = fileName + '.root'
#outputFileSuffix = '_ParticleToParticleTruth.pt'
outputFileSuffix = '_NeutrinoToParticleTruth_PosBothWay_NegBothWay_100Hits_wGenTruth_noBothNullEdgeCut_0_1_2.pt'

treeFile = uproot.open(inputFileName)
tree = treeFile['ccnuselection/ccnusel']
branches = tree.arrays()

In [3]:
#############################
# Get event-level stuff
#############################
run = np.array(branches['Run'])
subrun = np.array(branches['SubRun'])
event = np.array(branches['Event'])

recoNuVertexX = branches['RecoNuVtxX']
recoNuVertexY = branches['RecoNuVtxY']
recoNuVertexZ = branches['RecoNuVtxZ']
    
#############################
# Get pfp-level stuff - these cannot be numpy arrays...
#############################
trackShowerScore_main = branches['RecoPFPTrackShowerScore']
nHits_main = branches['RecoPFPRecoNHits']
charge_main = branches['RecoPFPRecoCharge']
vertexX_main = branches['RecoPFPRecoVertexX']
vertexY_main = branches['RecoPFPRecoVertexY']
vertexZ_main = branches['RecoPFPRecoVertexZ']
trackEndX_main = branches['RecoTrackRecoEndX']
trackEndY_main = branches['RecoTrackRecoEndY']
trackEndZ_main = branches['RecoTrackRecoEndZ']
showerDirX_main = branches['RecoShowerRecoDirX']  # not the best direction estimate, placeholder
showerDirY_main = branches['RecoShowerRecoDirY']
showerDirZ_main = branches['RecoShowerRecoDirZ']
ivysaurusMuon_main = branches['RecoPFPIvysaurusMuon']
ivysaurusProton_main = branches['RecoPFPIvysaurusProton']
ivysaurusPion_main = branches['RecoPFPIvysaurusPion']
ivysaurusElectron_main = branches['RecoPFPIvysaurusElectron']
ivysaurusPhoton_main = branches['RecoPFPIvysaurusPhoton']
wiggliness_main = branches['RecoTrackDeflecAngleSD']
trackLength_main = branches['RecoTrackLength']
displacement_main = branches['RecoShowerPandrizzleDisplacement']
dca_main = branches['RecoShowerPandrizzleDCA']
recoGeneration_main = branches['RecoPFPRecoGeneration']
recoSelf_main = branches['RecoPFPSelf']
pathwayLength_main = branches['RecoShowerPandrizzlePathwayLengthMin']
pathwayScatteringAngle2D_main = branches['RecoShowerPandrizzleMaxShowerStartPathwayScatteringAngle2D']
nShowerStartHits_main = branches['RecoShowerPandrizzleMaxNPostShowerStartHits']
showerScatterAngle_main = branches['RecoShowerPandrizzleMaxPostShowerStartScatterAngle']
nuVertexEnergyAsymmetry_main = branches['RecoShowerPandrizzleMaxPostShowerStartNuVertexEnergyAsymmetry']
showerStartEnergyAsymmetry_main = branches['RecoShowerPandrizzleMaxPostShowerStartShowerStartEnergyAsymmetry']
nuVertexEnergyWeightedMeanRadialDistance_main = branches['RecoShowerPandrizzleMaxPostShowerStartNuVertexEnergyWeightedMeanRadialDistance']
showerMoliereRadius_main = branches['RecoShowerPandrizzleMinPostShowerStartShowerStartMoliereRadius']
showerOpeningAngle_main = branches['RecoShowerPandrizzleMaxPostShowerStartOpeningAngle']
foundHitRatio_main = branches['RecoShowerPandrizzleMaxFoundHitRatio']
initialGapSize_main = branches['RecoShowerPandrizzleMaxInitialGapSize']
largestProjectedGapSize_main = branches['RecoShowerPandrizzleMinLargestProjectedGapSize']
nViewsWithAmbiguousHits_main = branches['RecoShowerPandrizzleNViewsWithAmbiguousHits']
ambiguousHitMaxUnaccountedEnergy_main = branches['RecoShowerPandrizzleAmbiguousHitMaxUnaccountedEnergy']

# True information.. (for cheating)
pfpTrueMomX_main = branches['RecoPFPTrueMomX']
pfpTrueMomY_main = branches['RecoPFPTrueMomY']
pfpTrueMomZ_main = branches['RecoPFPTrueMomZ']
pfpTruePDG_main = branches['RecoPFPTruePDG']

#############################
# Network truth - these cannot be numpy arrays...
#############################
trueVisibleGeneration_main = branches['RecoPFPTrueVisibleGeneration']
trueTrackID_main = branches['RecoPFPTrueTrackID']
trueVisibleParentTrackID_main = branches['RecoPFPTrueVisibleParentTrackID']

#############################
# How many entries are we working with?
#############################
nEntries = run.shape[0]
print('We are working with:', nEntries, 'entries')

We are working with: 287653 entries


In [ ]:
#############################
# Let's create our graphs!
#############################

data_set_pos = []
data_set_neg = []
data_set_FC = []

modeDict = {
    "ADD_NEUTRINO"                 : True,
    "CHEAT_DIRECTION"              : True,
    "CHEAT_PID"                    : True, 
    "MAKE_PARTICLE_PARTICLE_LINKS" : True,
    "EDGE_FRACTION"                : 0.8,
    "DO_NORMALISATION"             : True,
    "IS_PRIMARY_TRAINING"          : True,
    "IS_HIGHER_TIER_TRAINING"      : False
}

for iEvent in range(nEntries) : 
    
    #print(iEvent)
    #print('trueVisibleParentTrackID_main[iEvent]', trueVisibleParentTrackID_main[iEvent])
    
    eventDict = {
        "recoNuVertexX"                            : recoNuVertexX[iEvent], \
        "recoNuVertexY"                            : recoNuVertexY[iEvent], \
        "recoNuVertexZ"                            : recoNuVertexZ[iEvent], \
        "nParticles"                               : np.array(pfpTruePDG_main[iEvent]).shape[0], \
        "trackShowerScore"                         : np.array(trackShowerScore_main[iEvent], dtype='f'), \
        "nHits"                                    : np.array(nHits_main[iEvent], dtype='f'), \
        "charge"                                   : np.array(charge_main[iEvent], dtype='f'), \
        "vertexX"                                  : np.array(vertexX_main[iEvent], dtype='f'), \
        "vertexY"                                  : np.array(vertexY_main[iEvent], dtype='f'), \
        "vertexZ"                                  : np.array(vertexZ_main[iEvent], dtype='f'), \
        "trackEndX"                                : np.array(trackEndX_main[iEvent], dtype='f'), \
        "trackEndY"                                : np.array(trackEndY_main[iEvent], dtype='f'), \
        "trackEndZ"                                : np.array(trackEndZ_main[iEvent], dtype='f'), \
        "showerDirX"                               : np.array(showerDirX_main[iEvent], dtype='f'), \
        "showerDirY"                               : np.array(showerDirY_main[iEvent], dtype='f'), \
        "showerDirZ"                               : np.array(showerDirZ_main[iEvent], dtype='f'), \
        "ivysaurusMuon"                            : np.array(ivysaurusMuon_main[iEvent], dtype='f'), \
        "ivysaurusProton"                          : np.array(ivysaurusProton_main[iEvent], dtype='f'), \
        "ivysaurusPion"                            : np.array(ivysaurusPion_main[iEvent], dtype='f'), \
        "ivysaurusElectron"                        : np.array(ivysaurusElectron_main[iEvent], dtype='f'), \
        "ivysaurusPhoton"                          : np.array(ivysaurusPhoton_main[iEvent], dtype='f'), \
        "trackLength"                              : np.array(trackLength_main[iEvent], dtype='f'), \
        "displacement"                             : np.array(displacement_main[iEvent], dtype='f'), \
        "dca"                                      : np.array(dca_main[iEvent], dtype='f'), \
        "isNeutrinoPDG"                            : np.zeros(np.array(pfpTruePDG_main[iEvent]).shape), \
        "nuVertexEnergyAsymmetry"                  : np.array(nuVertexEnergyAsymmetry_main[iEvent], dtype='f'), \
        "nuVertexEnergyWeightedMeanRadialDistance" : np.array(nuVertexEnergyWeightedMeanRadialDistance_main[iEvent], dtype='f'), \
        "trueTrackID"                              : np.array(trueTrackID_main[iEvent]), \
        "trueVisibleParentTrackID"                 : np.array(trueVisibleParentTrackID_main[iEvent]), \
        "trueMomX"                                 : np.array(pfpTrueMomX_main[iEvent]), \
        "trueMomY"                                 : np.array(pfpTrueMomY_main[iEvent]), \
        "trueMomZ"                                 : np.array(pfpTrueMomZ_main[iEvent]), \
        "truePDG"                                  : np.array(pfpTruePDG_main[iEvent]), \
        "trueVisibleGeneration"                    : np.array(trueVisibleGeneration_main[iEvent])
    }
        
    data_pos, data_neg, data_FC, pfpIndex = GraphBuilder.GraphBuilder(eventDict, modeDict)

    if ((data_pos.num_edges == 0) and (data_neg.num_edges == 0)) :
        continue
        
    data_set_pos.append(data_pos)
    data_set_neg.append(data_neg)
    data_set_FC.append(data_FC)


In [ ]:
torch.save(data_set_pos, str('data_set_pos' + outputFileSuffix))
torch.save(data_set_neg, str('data_set_neg' + outputFileSuffix))
torch.save(data_set_FC, str('data_set_FC' + outputFileSuffix))

In [ ]:
print()